Example timeseries plot (unstandardized)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# SCENARIO 1
s1 = ['vp4s1d1', 'vp5s1d1', 'vp6s1d1', 'vp7s1d1', 'vp8s1d1', 'vp10s1d1', 'vp11s1d1', 'vp12s1d1', 'vp13s1d1', 'vp15s1d1', 'vp16s1d1', 'vp17s1d1', 'vp22s1d1', 'vp23s1d1', 'vp24s1d1', 'vp25s1d1', 'vp27s1d1', 'vp28s1d1', 'vp4s1d2', 'vp5s1d2', 'vp6s1d2', 'vp7s1d2', 'vp8s1d2', 'vp10s1d2', 'vp11s1d2', 'vp12s1d2', 'vp13s1d2', 'vp15s1d2', 'vp16s1d2', 'vp17s1d2', 'vp22s1d2', 'vp23s1d2', 'vp24s1d2', 'vp25s1d2', 'vp27s1d2', 'vp28s1d2']

totpd = []
for df_name in s1:
    df = globals()[df_name] 
    pd = df['field.eyeTracker.filteredPupilDiameter']
    totpd.extend(pd)

# 1.Outliers - 3 standard deviations above the mean    
mean_pd = np.mean(totpd)
sdev_pd = np.std(totpd)
threshold = mean_pd + 3 * sdev_pd

# 2. Initialize empty lists to store 5s worth time data per intervention phase
bef_int = []
dur_int = []
aft_int = []

fig, ax = plt.subplots(figsize=(10, 6))

# 3. Iterate through s1 data and change x-axis
for df_name in s1:
    df = globals()[df_name]
    time = df['ser_rostime_0s']
    pd = df['field.eyeTracker.filteredPupilDiameter']
    intervention_flag = df['field.hmiBackgroundState']
    
    # Intervention Condition for S1
    event_indices = intervention_flag[intervention_flag == 4].index
    if len(event_indices) > 0:
        start_time = time[event_indices[0]]
        end_time = start_time + 5  # Trim df
        bef_int.append(pd[(time >= start_time - 5) & (time < start_time)])
        dur_int.append(pd[(time >= start_time) & (time <= end_time)])
        aft_int.append(pd[(time > end_time) & (time <= end_time + 5)])
        
        # x-axis start from 0
        ax.plot(time - start_time, pd, color='lightgrey')  # Plot light grey line

# Plot overall representative behavior
overall_pd = np.mean([df['field.eyeTracker.filteredPupilDiameter'] for df_name in s1], axis=0)
ax.plot(time - start_time, overall_pd, color='blue', label='Overall Behavior')

ax.axhline(y=threshold, color='r', linestyle='--', label='Outlier Threshold')
y_axis_min = 0  
y_axis_max = 0.015

# 4. Plot pupil diameter vs time
ax.set_xlim([-5, 10])  # timelimit
ax.set_ylim([y_axis_min, y_axis_max])  
ax.set_xticks(np.arange(-5, 11, 1))
ax.set_xlabel('Time (in sec)')
ax.set_ylabel('Pupil Diameter [mm]')
ax.set_title('Pupil Diameter in Scenario 1')
ax.legend() 
plt.tight_layout()
plt.show()


Example time series plot (standardized)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# SCENARIO 1
s1 = ['vp4s1d1', 'vp5s1d1', 'vp6s1d1', 'vp7s1d1', 'vp8s1d1', 'vp10s1d1', 'vp11s1d1', 'vp12s1d1', 'vp13s1d1', 'vp15s1d1', 'vp16s1d1', 'vp17s1d1', 'vp22s1d1', 'vp23s1d1', 'vp24s1d1', 'vp25s1d1', 'vp27s1d1', 'vp28s1d1', 'vp4s1d2', 'vp5s1d2', 'vp6s1d2', 'vp7s1d2', 'vp8s1d2', 'vp10s1d2', 'vp11s1d2', 'vp12s1d2', 'vp13s1d2', 'vp15s1d2', 'vp16s1d2', 'vp17s1d2', 'vp22s1d2', 'vp23s1d2', 'vp24s1d2', 'vp25s1d2', 'vp27s1d2', 'vp28s1d2']

for df_name in s1:
    df = globals()[df_name]
    pd_data = df['field.eyeTracker.filteredPupilDiameter']
    baseline = pd_data.mean()
    std_dev = pd_data.std()
    
    df['Baseline'] = baseline
    df['StdDev'] = std_dev

    # print("Baseline for", df_name, ":", baseline)
    # print("Standard Deviation for", df_name, ":", std_dev)

z_scores = []

# Get the maximum length of time series data
max_length = max(len(globals()[df_name]['ser_rostime_0s']) for df_name in s1)
print("Maximum length of time series data:", max_length)

# Iterate through each participant df
for df_name in s1:
    df = globals()[df_name]
    time_data = df['ser_rostime_0s']
    pd_data = df['field.eyeTracker.filteredPupilDiameter']
    
    # Print available column names for reference
    print("Available columns in dataframe:", df.columns)
    
    # Identify baseline and standard deviation column names
    baseline_column = 'Baseline'  # Replace 'Baseline' with the actual column name
    std_dev_column = 'StdDev'  # Replace 'StdDev' with the actual column name
    
    # Check if baseline and standard deviation columns are present
    if baseline_column not in df.columns or std_dev_column not in df.columns:
        print("Baseline or StdDev columns not found in dataframe", df_name)
        continue
    
    # Identify baseline and standard deviation
    baseline = df[baseline_column].mean()
    std_dev = df[std_dev_column].mean()
    print("Baseline:", baseline)
    print("Standard Deviation:", std_dev)
    
    # Calculate z-scores for each participant
    z_score = (pd_data - baseline) / std_dev
    print("Z-scores for", df_name, ":", z_score)
    
    # Pad z-scores array with NaN values to match the length of the longest array
    padded_z_score = np.pad(z_score, (0, max_length - len(z_score)), mode='constant', constant_values=np.nan)
    
    # Append padded z-scores array to the list
    z_scores.append(padded_z_score)

# Convert the list of padded z-scores arrays to a numpy array
z_scores_concatenated = np.vstack(z_scores)
# print("Shape of concatenated z-scores array:", z_scores_concatenated.shape)
# print("Concatenated z-scores array:", z_scores_concatenated)

mean_z_scores = np.nanmean(z_scores_concatenated, axis=0)
sem_z_scores = np.nanstd(z_scores_concatenated, axis=0) / np.sqrt(len(s1))
time_data_truncated = np.linspace(-5, 10, max_length)

fig, ax = plt.subplots(figsize=(10, 6))
for z_score in z_scores:
    ax.plot(time_data_truncated, z_score, color='lightgrey', zorder=1)

# Plot representative lineplot with 95% CI with higher z-order
ax.plot(time_data_truncated, mean_z_scores, color='blue', label='Representative Pupil Diameter', zorder=2)
ax.fill_between(time_data_truncated, mean_z_scores - sem_z_scores * 1.96, mean_z_scores + sem_z_scores * 1.96, color='orange', alpha=0.3, label='95% CI', zorder=3)
ax.set_xlim(-5, 10)
ax.set_ylim(-5, 15)  # Adjust y-axis limits
ax.set_xlabel('Time (in sec)')
ax.set_ylabel('Standardized Pupil Diameter')
ax.set_title('Scenario 1')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()
